In [10]:
import cv2
import numpy as np
from PIL import Image 
#import Image
import pyttsx3
import pytesseract
pytesseract.pytesseract.tesseract_cmd = 'C:/Program Files (x86)/Tesseract-OCR/tesseract'
#import image
DEBUG=0
image = cv2.imread('C:/Users/hi/AppData/Local/Programs/Python/Python36/models/object_detection/test_images/image90.jpg')
#im=Image.open('start.jpg')
img_y = len(image)
img_x = len(image[0])
print(img_x,img_y)
#w2,h2=im.size
#print(w2,h2)
gray = cv2.cvtColor(image,cv2.COLOR_BGR2GRAY)
#cv2.imshow('gray', gray)
#cv2.waitKey(0)
def ii(xx, yy):
    global image, img_y, img_x
    if yy >= img_y or xx >= img_x:
        #print "pixel out of bounds ("+str(y)+","+str(x)+")"
        return 0
    pixel = image[yy][xx]
    return 0.30 * pixel[2] + 0.59 * pixel[1] + 0.11 * pixel[0]
def connected(contour):
    first = contour[0][0]
    last = contour[len(contour) - 1][0]
    return abs(first[0] - last[0]) <= 1 and abs(first[1] - last[1]) <= 1

"""def keep(contour):
    return keep_box(contour) #and connected(contour)

"""

# Whether we should keep the containing box of this
# contour based on it's shape
"""def keep_box(contour):
    xx, yy, w_, h_ = cv2.boundingRect(contour)

    # width and height need to be floats
    w_ *= 1.0
    h_ *= 1.0

    # Test it's shape - if it's too oblong or tall it's
    # probably not a real character
    if w_ / h_ < 0.1 or w_ / h_ > 10:
        if DEBUG:
            print "\t Rejected because of shape: (" + str(xx) + "," + str(yy) + "," + str(w_) + "," + str(h_) + ")" + \
                  str(w_ / h_)
        return False
    
    # check size of the box
    if ((w_ * h_) > ((img_x * img_y) / 5)) or ((w_ * h_) < 15):
        if DEBUG:
            print "\t Rejected because of size"
        return False

    return True"""


#binary
ret,thresh = cv2.threshold(gray,127,255,cv2.THRESH_BINARY_INV)
#cv2.imshow('second', thresh)
#cv2.waitKey(0)
 
#dilation
kernel = np.ones((1,1), np.uint8)
img_dilation = cv2.dilate(thresh, kernel, iterations=1)
#ret1,thresh1 = cv2.threshold(img_dilation,10,255,cv2.THRESH_OTSU)
#cv2.imshow('dilated', img_dilation)

img1=cv2.bitwise_not(img_dilation)
#cv2.imshow('inv', img1)
#cv2.waitKey(0)
#find contours

im2,ctrs, hier = cv2.findContours(img_dilation.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
#print ctrs,hier
#sort contours
sorted_ctrs = sorted(ctrs, key=lambda ctr: cv2.boundingRect(ctr)[0])
keepers=[]
#print(sorted_ctrs)
for i, ctr in enumerate(sorted_ctrs):
    """if(i==len(sorted_ctrs)-5):
     	break;"""
    #print(ctr)
    # Get bounding box
    #if (w > 15 and h > 15):
    
    x, y, w, h = cv2.boundingRect(ctr)
    if((w<img_x-8) and (h<img_y-8)) and (w > 5 and h > 5):
        #and (w<140 and h<140):
        keepers.append([ctr, [x, y, w, h]])
        #print(x,y,w,h)
        roi = image[y:y+h, x:x+w]
        cv2.rectangle(image,(x,y),( x + (w), y + (h) ),(0,255,0),2)
        if(w > 5 and h > 5):
            cv2.imwrite('C:/Users/hi/AppData/Local/Programs/Python/Python36/models/pics/{}.png'.format(i), roi)
#ret2,thresh2 = cv2.threshold(image,10,255,cv2.THRESH_OTSU)
new_image = im2.copy()
new_image.fill(255)
cv2.imwrite("C:/Users/hi/AppData/Local/Programs/Python/Python36/models/filledge.jpg",new_image)
boxes = []

# For each box, find the foreground and background intensities
for index_, (contour_, box) in enumerate(keepers):

    # Find the average intensity of the edge pixels to
    # determine the foreground intensity
    fg_int = 0.0
    for p in contour_:
        fg_int += ii(p[0][0], p[0][1])

    fg_int /= len(contour_)
    if DEBUG:
        print("FG Intensity for #%d = %d" % (index_, fg_int))

    # Find the intensity of three pixels going around the
    # outside of each corner of the bounding box to determine
    # the background intensity
    x_, y_, width, height = box
    bg_int = \
        [
            # bottom left corner 3 pixels
            ii(x_ - 1, y_ - 1),
            ii(x_ - 1, y_),
            ii(x_, y_ - 1),

            # bottom right corner 3 pixels
            ii(x_ + width + 1, y_ - 1),
            ii(x_ + width, y_ - 1),
            ii(x_ + width + 1, y_),

            # top left corner 3 pixels
            ii(x_ - 1, y_ + height + 1),
            ii(x_ - 1, y_ + height),
            ii(x_, y_ + height + 1),

            # top right corner 3 pixels
            ii(x_ + width + 1, y_ + height + 1),
            ii(x_ + width, y_ + height + 1),
            ii(x_ + width + 1, y_ + height)
        ]

    # Find the median of the background
    # pixels determined above
    bg_int = np.median(bg_int)

    if DEBUG:
        print("BG Intensity for #%d = %s" % (index_, repr(bg_int)))

    # Determine if the box should be inverted
    if fg_int >= bg_int:
        fg = 255
        bg = 0
    else:
        fg = 0
        bg = 255

        # Loop through every pixel in the box and color the
        # pixel accordingly
    for x in range(x_, x_ + width):
        for y in range(y_, y_ + height):
            if y >= img_y or x >= img_x:
                if DEBUG:
                    print("pixel out of bounds (%d,%d)" % (y, x))
                continue
            if ii(x, y) > fg_int:
                new_image[y][x] = bg
            else:
                new_image[y][x] = fg

# blur a bit to improve ocr accuracy
#new_image = cv2.blur(new_image, (1, 1))
###kernel = np.ones((1,1), np.uint8)
#new_image = cv2.dilate(new_image, kernel, iterations=1)
###new_image = cv2.blur(new_image, (1, 1))
cv2.imwrite('C:/Users/hi/AppData/Local/Programs/Python/Python36/models/text.jpg', new_image)
#print(pytesseract.image_to_string(Image.open('C:/Users/hi/AppData/Local/Programs/Python/Python36/models/text.jpg')))
engine = pyttsx3.init()
img11=Image.open('C:/Users/hi/AppData/Local/Programs/Python/Python36/models/text.jpg')
print(pytesseract.image_to_string(img11))
engine.say(pytesseract.image_to_string(img11))
engine.runAndWait()
cv2.imshow('marked areas',image)
#cv2.imshow('marked',thresh2)
img1=cv2.bitwise_not(image)
#cv2.imshow('inv_img', img1)
#cv2.imwrite('marked.png',image)
#cv2.imshow('roi1.png',roi1)
cv2.waitKey(0)

640 480



-1

In [9]:
import cv2
cap = cv2.VideoCapture(1)

while True:

    ret, frame = cap.read()

    if ret == True:

        #gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

        cv2.imshow('frame',frame)


        if cv2.waitKey(30) & 0xFF == ord('q'):
            cv2.imwrite('C:/Users/hi/AppData/Local/Programs/Python/Python36/models/object_detection/test_images/image90.jpg',frame)
            break

    else:
        break

cap.release()
cv2.destroyAllWindows()